# Web Scraping
# World's Top Billionaires 2021

In [1]:
#Importar librerías
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import csv
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

### Funciones:

Primero se definirán las funciones a utilizar. Se necesitan:

-Función que dé click en la primera fila de la tabla para quitar el despliegue que genera la página por defecto, a la cual se le pasa por parámetro el full xpath (clicks(datos_xpath)).

-Función que extraiga los datos de las tablas presentes en la página. Su parámetro es una lista que contiene las columnas que leerá ya que dependiendo de la columna cambia el xpath (extraer_datos(lista)).

-Función para organizar los datos por tipo, ya sea Rank, Name,.... Se le pasa como parámetros la lista de los datos extraídos, de qué posición inicia, y hasta qué posición considerará (organiza(E,inicio,fin)).

In [2]:
#hace click para quitar o colocar despliegue de cada item de la tabla
def clicks(datos_xpath):
    button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, datos_xpath)))
    button.click()


#### Ahora se necesita una función que  recorra la tabla para extraer los datos. Para ello se necesitan los full Xpath y ver la relación entre estos, luego se emplean ciclos for para recorrerlo


#### Full Xpath 

Se mueve de la siguiente forma:

'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[ j (se mueve entre distintas tablas de la página)]/div[i (se mueve entre las filas de la página)]/div[k (selecciona las columnas)]

- Con /div al final: Extrae Name,NetWorth,Age,Souce
- Sin /div al final: Extrae Rank, Country/territory,Industry

Ejemplo:

- Para la primera fila:

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2] #rank

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div #name

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[3]/div #NetWorks

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[4]/div #age

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[5] #country

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[6]/div #source

/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[7] #industry

In [3]:
#Como el xpath es una cadena (extraí full xpath), se identifica en cuales posiciones debe cambiar, así se van construyendo
#   los diferentes xpath que se necesitan para extraer la data
#Para extraer Rank, Country/territory,Industry: lista2=[1,5,7]
#Para extraer: Name,NetWorth,Age,Souce: lista1=[2,3,4,6]

def extraer_datos(lista):
    lista2=[1,5,7]
    lista1=[2,3,4,6]
    if lista==lista1:
        f=']/div' #Name,NetWorth,Age,Souce
    if lista==lista2: #lista2=[1,5,7]
        f=']' #Rank, Country/territory,Industry
        
    
    E=[]
    for k in lista: #selecciona las columnas correspondientes a Rank, Country/territory,Industry
        for j in range(1,28,2): #se mueve entre distintas tablas de la página
        
        #construcción de la cadena del full xpath (a excepción de la i, esta se hace internamente dentro del condicional):
            a='/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div['
            b=']/div['
            c=']/div['
            g=str(k)
            e=str(j)
        
        
        #para la primera tabla
            if j==1:
                for i in range(2,18): #se mueve entre las filas de la página
                    if i!=3:
                        d=str(i)
                        cadena=a+e+b+d+c+g+f
                        extraer=list(map(lambda x: x.text, driver.find_elements_by_xpath(cadena)))
                        E.append(extraer)
        #para las que estrén entre (1,27) (intervalo abierto)
            if j>1 and j<27:
                for i in range(2,17): #se mueve entre las filas de la página
                    d=str(i)
                    cadena=a+e+b+d+c+g+f
                    extraer=list(map(lambda x: x.text, driver.find_elements_by_xpath(cadena)))
                    E.append(extraer)
                
        #para la última tabla
            if j==27:
                for i in range(2,7): #se mueve entre las filas de la página
                    d=str(i)
                    cadena=a+e+b+d+c+g+f
                    extraer=list(map(lambda x: x.text, driver.find_elements_by_xpath(cadena)))
                    E.append(extraer)
    return E   
      

In [4]:
#Función para organizar lo extraído: 
def organiza(E,inicio,fin):
        result=[]
        for i in range(inicio,fin):
            result.append(E[i][0])
        return result

In [5]:
#Esta función extrae los datos de página en página
def salta():
    for pagina in range(0,13):

        #utilizando la función extraer_datos()
        lista2=[1,5,7]
        lista1=[2,3,4,6]
        E2=extraer_datos(lista2) #Extrae Rank, Country/territory,Industry
        E1=extraer_datos(lista1) #Extrae Name,NetWorth,Age,Souce
    
        #aplicando función organiza para acomodar los datos por tipo:
        #en E1 se tiene Name,NetWorth,Age,Souce
        x=0
        y=200 #Estoy sacando 200 filas, que es lo que contiene cada página
        Name=organiza(E1,x,y)
        NetWorth=organiza(E1,y,2*y)
        Age=organiza(E1,2*y,3*y)
        Source=organiza(E1,3*y,4*y)
    
        #aplicando función organiza para acomodar los datos por tipo:
        #en E2 se tiene Rank, Country/territory,Industry
        Rank=organiza(E2,x,y)
        Country=organiza(E2,y,2*y)
        Industry=organiza(E2,2*y,3*y)
    
        #ahora guardar los datos en un pandas
        if pagina==0: #aquí se crea el primer dataframe de la primera página
            DF=pd.DataFrame(list(zip(Rank,Name,NetWorth,Age,Country,Source,Industry)),columns=['Rank','Name','NetWorth','Age','Country','Source','Industry'])
        else: #aquí se hace con las siguientes
            df=pd.DataFrame(list(zip(Rank,Name,NetWorth,Age,Country,Source,Industry)),columns=['Rank','Name','NetWorth','Age','Country','Source','Industry'])
            DF=pd.concat([DF,df], axis=0)
        
        
        #time.sleep(random.randint(10,20)) #tiempo de espera entre ejecución aleatorio entre 10 y 20 segundos
        
        #salta a la siguiente página
        xpath_cambio_de_pagina='/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[27]/div[7]/div[2]/button[2]/div'
        clicks(xpath_cambio_de_pagina)
        
    return DF

### Ejecución utilizando las funciones anteriores:

In [6]:

#Abre la página en el buscador. Aquí se usa Chrome:

driver = webdriver.Chrome(r'chromedriver.exe')
driver.get("https://www.forbes.com/billionaires/list/50/#version:realtime")

df=salta() #usar función que extrae datos de página en página

df.to_csv('valores.csv', header=False, index=False) #ahora para guardar todo en un csv 


driver.close() #Cierra la página en el buscador